### Univariate LSTM Models

In [1]:
import numpy as np
from numpy import array
from numpy import hstack


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [3]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


#### 1. Vanila LSTM

In [4]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X.shape)
print(y.shape)

(6, 3, 1)
(6,)


In [5]:
# LSTM model
model = Sequential()
model.add(layer=LSTM(
    units=50, activation='relu', input_shape=(n_steps, n_features))
)
model.add(layer=Dense(units=1))
model.compile(optimizer='adam', loss='mse')

In [6]:
# fit model
model.fit(x=X, y=y, epochs=200, verbose=True)

Train on 6 samples
Epoch 1/200
6/6 [==============================] - 1s 210ms/sample - loss: 4824.2632
Epoch 2/200
6/6 [==============================] - 0s 579us/sample - loss: 4751.4565
Epoch 3/200
6/6 [==============================] - 0s 480us/sample - loss: 4679.5664
Epoch 4/200
6/6 [==============================] - 0s 587us/sample - loss: 4608.3076
Epoch 5/200
6/6 [==============================] - 0s 569us/sample - loss: 4537.4609
Epoch 6/200
6/6 [==============================] - 0s 549us/sample - loss: 4466.8022
Epoch 7/200
6/6 [==============================] - 0s 537us/sample - loss: 4396.1309
Epoch 8/200
6/6 [==============================] - 0s 539us/sample - loss: 4325.3760
Epoch 9/200
6/6 [==============================] - 0s 565us/sample - loss: 4254.5098
Epoch 10/200
6/6 [==============================] - 0s 656us/sample - loss: 4183.5181
Epoch 11/200
6/6 [==============================] - 0s 653us/sample - loss: 4112.4102
Epoch 12/200
6/6 [==========================

Epoch 97/200
6/6 [==============================] - 0s 645us/sample - loss: 83.6953
Epoch 98/200
6/6 [==============================] - 0s 581us/sample - loss: 82.4925
Epoch 99/200
6/6 [==============================] - 0s 730us/sample - loss: 81.3249
Epoch 100/200
6/6 [==============================] - 0s 600us/sample - loss: 80.1830
Epoch 101/200
6/6 [==============================] - 0s 541us/sample - loss: 79.0591
Epoch 102/200
6/6 [==============================] - 0s 639us/sample - loss: 77.9490
Epoch 103/200
6/6 [==============================] - 0s 585us/sample - loss: 76.8410
Epoch 104/200
6/6 [==============================] - 0s 639us/sample - loss: 75.7299
Epoch 105/200
6/6 [==============================] - 0s 577us/sample - loss: 74.6120
Epoch 106/200
6/6 [==============================] - 0s 625us/sample - loss: 73.4854
Epoch 107/200
6/6 [==============================] - 0s 611us/sample - loss: 72.3495
Epoch 108/200
6/6 [==============================] - 0s 563us/sample

Epoch 194/200
6/6 [==============================] - 0s 581us/sample - loss: 2.6445
Epoch 195/200
6/6 [==============================] - 0s 567us/sample - loss: 2.5086
Epoch 196/200
6/6 [==============================] - 0s 564us/sample - loss: 2.3769
Epoch 197/200
6/6 [==============================] - 0s 609us/sample - loss: 2.2506
Epoch 198/200
6/6 [==============================] - 0s 551us/sample - loss: 2.1302
Epoch 199/200
6/6 [==============================] - 0s 591us/sample - loss: 2.0159
Epoch 200/200
6/6 [==============================] - 0s 600us/sample - loss: 1.9074


In [7]:
# demo predictions
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
print(x_input)
print(x_input.shape)
yhat = model.predict(tf.cast(x_input, tf.float32), verbose=True)
print(yhat)
print(yhat.shape)

[[[70]
  [80]
  [90]]]
(1, 3, 1)
1/1 [==============================] - 0s 148ms/sample
[[103.32138]]
(1, 1)


#### 2. Stacked LSTM

In [8]:
# Stacked model
n_features = 1
model = Sequential()
model.add(layer=LSTM(
    units=50, activation='relu', return_sequences=True,
    input_shape=(n_steps, n_features))
)
model.add(layer=LSTM(units=50, activation='relu'))
model.add(layer=Dense(units=n_features))
model.compile(optimizer='adam', loss='mse')

In [9]:
# fit model
model.fit(x=X, y=y, epochs=200, verbose=True)

Train on 6 samples
Epoch 1/200
6/6 [==============================] - 2s 318ms/sample - loss: 4561.5957
Epoch 2/200
6/6 [==============================] - 0s 897us/sample - loss: 4531.4551
Epoch 3/200
6/6 [==============================] - 0s 853us/sample - loss: 4502.1660
Epoch 4/200
6/6 [==============================] - 0s 884us/sample - loss: 4473.0854
Epoch 5/200
6/6 [==============================] - 0s 857us/sample - loss: 4443.7437
Epoch 6/200
6/6 [==============================] - 0s 892us/sample - loss: 4413.6450
Epoch 7/200
6/6 [==============================] - 0s 962us/sample - loss: 4381.8750
Epoch 8/200
6/6 [==============================] - 0s 998us/sample - loss: 4347.6685
Epoch 9/200
6/6 [==============================] - 0s 1ms/sample - loss: 4310.2905
Epoch 10/200
6/6 [==============================] - 0s 1ms/sample - loss: 4269.1382
Epoch 11/200
6/6 [==============================] - 0s 1ms/sample - loss: 4223.4321
Epoch 12/200
6/6 [==============================] 

Epoch 99/200
6/6 [==============================] - 0s 900us/sample - loss: 3.6617
Epoch 100/200
6/6 [==============================] - 0s 937us/sample - loss: 3.5309
Epoch 101/200
6/6 [==============================] - 0s 896us/sample - loss: 3.3419
Epoch 102/200
6/6 [==============================] - 0s 945us/sample - loss: 3.1478
Epoch 103/200
6/6 [==============================] - 0s 936us/sample - loss: 2.9959
Epoch 104/200
6/6 [==============================] - 0s 998us/sample - loss: 2.9147
Epoch 105/200
6/6 [==============================] - 0s 948us/sample - loss: 2.9078
Epoch 106/200
6/6 [==============================] - 0s 891us/sample - loss: 2.9566
Epoch 107/200
6/6 [==============================] - 0s 889us/sample - loss: 3.0281
Epoch 108/200
6/6 [==============================] - 0s 930us/sample - loss: 3.0878
Epoch 109/200
6/6 [==============================] - 0s 961us/sample - loss: 3.1111
Epoch 110/200
6/6 [==============================] - 0s 922us/sample - loss: 

Epoch 197/200
6/6 [==============================] - 0s 888us/sample - loss: 2.2021
Epoch 198/200
6/6 [==============================] - 0s 873us/sample - loss: 2.1950
Epoch 199/200
6/6 [==============================] - 0s 868us/sample - loss: 2.1878
Epoch 200/200
6/6 [==============================] - 0s 898us/sample - loss: 2.1806


In [10]:
# demo prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
print(x_input)
print(x_input.shape)
yhat = model.predict(tf.cast(x_input, tf.float32), verbose=True)
print(yhat)
print(yhat.shape)

[[[70]
  [80]
  [90]]]
(1, 3, 1)
1/1 [==============================] - 0s 267ms/sample
[[104.897545]]
(1, 1)


#### 3. Bidirectional LSTM

In [11]:
# Bidirectional model
n_features = 1
model = Sequential()
model.add(layer=Bidirectional(layer=LSTM(
    units=50, activation='relu'),
    input_shape=(n_steps, n_features))
)
model.add(layer=Dense(units=n_features))
model.compile(optimizer='adam', loss='mse')

In [12]:
# fit model
model.fit(x=X, y=y, epochs=200, verbose=True)

Train on 6 samples
Epoch 1/200
6/6 [==============================] - 2s 309ms/sample - loss: 4519.6055
Epoch 2/200
6/6 [==============================] - 0s 862us/sample - loss: 4456.7188
Epoch 3/200
6/6 [==============================] - 0s 769us/sample - loss: 4394.1934
Epoch 4/200
6/6 [==============================] - 0s 757us/sample - loss: 4331.1387
Epoch 5/200
6/6 [==============================] - 0s 885us/sample - loss: 4267.5410
Epoch 6/200
6/6 [==============================] - 0s 843us/sample - loss: 4203.0581
Epoch 7/200
6/6 [==============================] - 0s 1ms/sample - loss: 4137.6460
Epoch 8/200
6/6 [==============================] - 0s 866us/sample - loss: 4071.2158
Epoch 9/200
6/6 [==============================] - 0s 882us/sample - loss: 4003.4949
Epoch 10/200
6/6 [==============================] - 0s 1ms/sample - loss: 3934.4773
Epoch 11/200
6/6 [==============================] - 0s 969us/sample - loss: 3864.4021
Epoch 12/200
6/6 [==============================

Epoch 98/200
6/6 [==============================] - 0s 793us/sample - loss: 1.4898
Epoch 99/200
6/6 [==============================] - 0s 952us/sample - loss: 1.6795
Epoch 100/200
6/6 [==============================] - 0s 816us/sample - loss: 1.6557
Epoch 101/200
6/6 [==============================] - 0s 805us/sample - loss: 1.4829
Epoch 102/200
6/6 [==============================] - 0s 842us/sample - loss: 1.3373
Epoch 103/200
6/6 [==============================] - 0s 832us/sample - loss: 1.3284
Epoch 104/200
6/6 [==============================] - 0s 815us/sample - loss: 1.4404
Epoch 105/200
6/6 [==============================] - 0s 1ms/sample - loss: 1.5808
Epoch 106/200
6/6 [==============================] - 0s 872us/sample - loss: 1.6583
Epoch 107/200
6/6 [==============================] - 0s 802us/sample - loss: 1.6307
Epoch 108/200
6/6 [==============================] - 0s 905us/sample - loss: 1.5136
Epoch 109/200
6/6 [==============================] - 0s 980us/sample - loss: 1.3

Epoch 196/200
6/6 [==============================] - 0s 965us/sample - loss: 0.2947
Epoch 197/200
6/6 [==============================] - 0s 914us/sample - loss: 0.2920
Epoch 198/200
6/6 [==============================] - 0s 914us/sample - loss: 0.2895
Epoch 199/200
6/6 [==============================] - 0s 1ms/sample - loss: 0.2870
Epoch 200/200
6/6 [==============================] - 0s 869us/sample - loss: 0.2845


In [13]:
# demo prediction
x_input = np.array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
print(x_input)
print(x_input.shape)
yhat = model.predict(tf.cast(x_input, tf.float32), verbose=True)
print(yhat)
print(yhat.shape)

[[[70]
  [80]
  [90]]]
(1, 3, 1)
1/1 [==============================] - 0s 302ms/sample
[[101.81503]]
(1, 1)


### Multivariate LSTM Models

In [14]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure 
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1)) 
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
print(dataset)
print(dataset.shape)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
(9, 3)


In [15]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape)
print(y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(7, 3, 2)
(7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


#### 1. Vanilla LSTM

In [16]:
# LSTM model
n_steps = 3
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
model = Sequential()
model.add(layer=LSTM(
    units=50, activation='relu', input_shape=(n_steps, n_features))
)
model.add(layer=Dense(units=1))
model.compile(optimizer='adam', loss='mse')

In [17]:
# fit model
model.fit(x=X, y=y, epochs=200, verbose=True)

Train on 7 samples
Epoch 1/200
7/7 [==============================] - 1s 173ms/sample - loss: 14998.3281
Epoch 2/200
7/7 [==============================] - 0s 543us/sample - loss: 14794.1270
Epoch 3/200
7/7 [==============================] - 0s 532us/sample - loss: 14583.5576
Epoch 4/200
7/7 [==============================] - 0s 535us/sample - loss: 14372.9395
Epoch 5/200
7/7 [==============================] - 0s 582us/sample - loss: 14169.9453
Epoch 6/200
7/7 [==============================] - 0s 526us/sample - loss: 13979.9404
Epoch 7/200
7/7 [==============================] - 0s 565us/sample - loss: 13803.7012
Epoch 8/200
7/7 [==============================] - 0s 591us/sample - loss: 13636.7402
Epoch 9/200
7/7 [==============================] - 0s 657us/sample - loss: 13472.4307
Epoch 10/200
7/7 [==============================] - 0s 566us/sample - loss: 13303.5186
Epoch 11/200
7/7 [==============================] - 0s 573us/sample - loss: 13123.6143
Epoch 12/200
7/7 [===============

Epoch 98/200
7/7 [==============================] - 0s 563us/sample - loss: 3.1656
Epoch 99/200
7/7 [==============================] - 0s 546us/sample - loss: 2.7672
Epoch 100/200
7/7 [==============================] - 0s 521us/sample - loss: 2.2830
Epoch 101/200
7/7 [==============================] - 0s 524us/sample - loss: 1.8307
Epoch 102/200
7/7 [==============================] - 0s 644us/sample - loss: 1.5478
Epoch 103/200
7/7 [==============================] - 0s 595us/sample - loss: 1.4275
Epoch 104/200
7/7 [==============================] - 0s 538us/sample - loss: 1.3681
Epoch 105/200
7/7 [==============================] - 0s 542us/sample - loss: 1.3388
Epoch 106/200
7/7 [==============================] - 0s 558us/sample - loss: 1.3563
Epoch 107/200
7/7 [==============================] - 0s 582us/sample - loss: 1.3751
Epoch 108/200
7/7 [==============================] - 0s 607us/sample - loss: 1.3151
Epoch 109/200
7/7 [==============================] - 0s 553us/sample - loss: 1

Epoch 196/200
7/7 [==============================] - 0s 618us/sample - loss: 0.0355
Epoch 197/200
7/7 [==============================] - 0s 512us/sample - loss: 0.0346
Epoch 198/200
7/7 [==============================] - 0s 499us/sample - loss: 0.0337
Epoch 199/200
7/7 [==============================] - 0s 566us/sample - loss: 0.0328
Epoch 200/200
7/7 [==============================] - 0s 565us/sample - loss: 0.0320


In [18]:
# demo prediction
x_input = np.array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
print(x_input)
print(x_input.shape)
yhat = model.predict(tf.cast(x_input, tf.float32), verbose=True)
print(yhat)
print(yhat.shape)

[[[ 80  85]
  [ 90  95]
  [100 105]]]
(1, 3, 2)
1/1 [==============================] - 0s 165ms/sample
[[204.92224]]
(1, 1)


### Multi-step LSTM Models
1. Vector Output Model

In [19]:
# multivariate data preparation
from numpy import array
from numpy import hstack
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))                 
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns                
dataset = hstack((in_seq1, in_seq2, out_seq))
print(dataset)
print(dataset.shape)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
(9, 3)


In [20]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(6, 3, 2) (6, 2)
[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]


In [21]:
# define model
model = Sequential()
model.add(layer=LSTM(
    units=100, activation='relu', return_sequences=True,
    input_shape=(n_steps_in, n_features))
)
model.add(layer=LSTM(units=100, activation='relu'))
model.add(layer=Dense(units=n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [22]:
model.fit(x=X, y=y, epochs=200, verbose=True)

Train on 6 samples
Epoch 1/200
6/6 [==============================] - 2s 352ms/sample - loss: 17066.3066
Epoch 2/200
6/6 [==============================] - 0s 1ms/sample - loss: 16753.7539
Epoch 3/200
6/6 [==============================] - 0s 1ms/sample - loss: 16463.0742
Epoch 4/200
6/6 [==============================] - 0s 1ms/sample - loss: 16170.1904
Epoch 5/200
6/6 [==============================] - 0s 1ms/sample - loss: 15872.0537
Epoch 6/200
6/6 [==============================] - 0s 1ms/sample - loss: 15563.0576
Epoch 7/200
6/6 [==============================] - 0s 1ms/sample - loss: 15229.2500
Epoch 8/200
6/6 [==============================] - 0s 1ms/sample - loss: 14862.8281
Epoch 9/200
6/6 [==============================] - 0s 1ms/sample - loss: 14452.2920
Epoch 10/200
6/6 [==============================] - 0s 1ms/sample - loss: 13991.1416
Epoch 11/200
6/6 [==============================] - 0s 1ms/sample - loss: 13472.6328
Epoch 12/200
6/6 [==============================] - 0

Epoch 100/200
6/6 [==============================] - 0s 954us/sample - loss: 2.5330
Epoch 101/200
6/6 [==============================] - 0s 1ms/sample - loss: 2.4791
Epoch 102/200
6/6 [==============================] - 0s 986us/sample - loss: 2.4018
Epoch 103/200
6/6 [==============================] - 0s 1ms/sample - loss: 2.3388
Epoch 104/200
6/6 [==============================] - 0s 955us/sample - loss: 2.3195
Epoch 105/200
6/6 [==============================] - 0s 973us/sample - loss: 2.3199
Epoch 106/200
6/6 [==============================] - 0s 959us/sample - loss: 2.3008
Epoch 107/200
6/6 [==============================] - 0s 1ms/sample - loss: 2.2604
Epoch 108/200
6/6 [==============================] - 0s 1ms/sample - loss: 2.2262
Epoch 109/200
6/6 [==============================] - 0s 1ms/sample - loss: 2.2109
Epoch 110/200
6/6 [==============================] - 0s 960us/sample - loss: 2.1922
Epoch 111/200
6/6 [==============================] - 0s 1ms/sample - loss: 2.1519
Epoc

Epoch 199/200
6/6 [==============================] - 0s 1ms/sample - loss: 0.1119
Epoch 200/200
6/6 [==============================] - 0s 1ms/sample - loss: 0.1082


In [23]:
# demo prediction
x_input = np.array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_steps, n_features))
print(x_input)
print(x_input.shape)
yhat = model.predict(tf.cast(x_input, tf.float32), verbose=True)
print(yhat)
print(yhat.shape)

[[[70 75]
  [80 85]
  [90 95]]]
(1, 3, 2)
1/1 [==============================] - 0s 291ms/sample
[[186.55225 208.21582]]
(1, 2)
